# PSET 1: Bottom-Up Synthesis

I follow Algorithm 1 in the BUSTLE paper:

> Odena, A. *et al.* BUSTLE: Bottom-Up Program Synthesis Through Learning-Guided Exploration. in *9th International Conference on Learning Representations*; 2021 May 3-7; Austria.

First, I import the required libraries.

In [6]:
import numpy as np
import itertools

# argument parser for command line arguments
import argparse

# import arithmetic module
# from arithmetic import *
from examples import examples
import config

First, I define variables as proxies for command-line arguments provided to the synthesizer.

In [ ]:
domain = "arithmetic"
examples_key = "addition"
examples = examples[examples_key]
max_weight = 3

I provide examples of arithmetic operations.

In [5]:
'''
ARTHIMETIC OPERATORS
This file contains Python classes that define the arithmetic operators for program synthesis.
'''

'''
CLASS DEFINITIONS
''' 

class IntegerValue:
    '''
    Class to represent an arithmetic value.
    '''
    def __init__(self, value):
        self.value = value
        self.type = int

class Add:
    '''
    Operator to add two numerical values.
    '''
    def __init__(self):
        self.arity = 2          # number of arguments of function
        self.weight = 1         # weight of function
        self.return_type = int  # return type of function

    def __call__(self, x, y):
        return x + y
    
    def str(x, y):
        return f"{x} + {y}"

class Subtract:
    '''
    Operator to subtract two numerical values.
    '''
    def __init__(self):
        self.arity = 2          # number of arguments of function
        self.weight = 1         # weight of function
        self.return_type = int  # return type of function

    def __call__(self, x, y):
        return x - y
    
    def str(x, y):
        return f"{x} - {y}"
    
class Multiply:
    '''
    Operator to multiply two numerical values.
    '''
    def __init__(self):
        self.arity = 2          # number of arguments of function
        self.weight = 1         # weight of function
        self.return_type = int  # return type of function

    def __call__(self, x, y):
        return x * y
    
    def str(x, y):
        return f"{x} * {y}" 

class Divide:
    '''
    Operator to divide two numerical values.
    '''
    def __init__(self):
        self.arity = 2          # number of arguments of function
        self.weight = 1         # weight of function
        self.return_type = int  # return type of function

    def __call__(self, x, y):
        try: # check for division by zero error
            return x / y
        except ZeroDivisionError:
            return None
    
    def str(x, y):
        return f"{x} / {y}"


'''
FUNCTION DEFINITIONS
''' 


'''
GLOBAL CONSTANTS
''' 

# define operators
arithmetic_operators = [Add(), Subtract(), Multiply(), Divide()]

I define input-output examples.

I define a function to determine observational equivalence.

In [ ]:
def observationally_equivalent(a, b):
    """
    Returns True if a and b are observationally equivalent, False otherwise.
    """

    pass

Next, I define the bottom-up synthesis algorithm.

In [2]:
# initialize program bank
program_bank = []

# iterate over each level
for i in range(1, max_level):

    # define level program bank
    level_program_bank = []

    for op in arithmetic_operators():

        break